In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, roc_auc_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
file_path = '/content/drive/MyDrive/dados/creditcard.csv'

df = pd.read_csv(file_path)

scaler = StandardScaler()
X = scaler.fit_transform(df.drop('Class', axis=1))
y = df['Class']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
def create_model(optimizer='adam', init='glorot_uniform'):
    model = Sequential()
    model.add(Input(shape=(X_train.shape[1],)))
    model.add(Dense(16, kernel_initializer=init, activation='relu'))
    model.add(Dense(8, kernel_initializer=init, activation='relu'))
    model.add(Dense(1, kernel_initializer=init, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

model = KerasClassifier(build_fn=create_model, verbose=0)

In [ ]:
param_grid = {
    'batch_size': [32, 64],
    'epochs': [10, 20],
    'optimizer': ['adam', 'rmsprop'],
    'init': ['glorot_uniform', 'normal']
}

grid = GridSearchCV(estimator=model, param_grid=param_grid, scoring='roc_auc', cv=3)
grid_result = grid.fit(X_train, y_train)

best_model = grid_result.best_estimator_
y_pred_best = best_model.predict(X_test)
y_pred_prob_best = best_model.predict_proba(X_test)[:, 1]

print(f"Melhor combinação de parâmetros: {grid_result.best_params_}")
print(f"Melhor AUC-ROC com Grid Search: {grid_result.best_score_}")

print("\nModelo Otimizado:")
print(classification_report(y_test, (y_pred_prob_best > 0.5).astype(int)))
print("AUC-ROC Otimizado:", roc_auc_score(y_test, y_pred_prob_best))

In [ ]:
original_model = create_model()
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
original_model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test), verbose=0, callbacks=[early_stopping])
y_pred_original = original_model.predict(X_test)
y_pred_prob_original = y_pred_original

auc_roc_original = roc_auc_score(y_test, y_pred_prob_original)
print(f"AUC-ROC Modelo Original: {auc_roc_original}")

print("\nComparação dos Resultados:")
print(f"AUC-ROC Modelo Original: {auc_roc_original}")
print(f"AUC-ROC Melhor Modelo Otimizado: {roc_auc_score(y_test, y_pred_prob_best)}")

**Explicações e Melhorias:**
Mais Neurônios e Camadas: O número de neurônios foi aumentado para dar ao modelo mais capacidade de aprender padrões complexos.

**Grid Search:** O GridSearchCV é usado para encontrar a melhor combinação de hiperparâmetros com base em uma grade de valores. A métrica roc_auc é usada para avaliar os modelos durante a busca.

**Validação Cruzada:** A validação cruzada (cv=3) é aplicada para melhorar a generalização do modelo.

**Early Stopping:** O early stopping foi ajustado para ter paciência maior, permitindo mais treinamento se necessário.

**Avaliação Completa:** Além da AUC-ROC, também é calculado o relatório de classificação completo para fornecer métricas adicionais como precisão, recall e F1-score.

Essas mudanças devem fornecer uma comparação mais robusta entre o modelo original e o modelo otimizado, além de permitir um ajuste mais preciso dos hiperparâmetros.